## Importing Required Packages

In [35]:
import os
import yaml
import json

from crewai.llm import LLM
from dotenv import load_dotenv

# Loading the environment variables
load_dotenv()

# Initializing the LLM
llm = LLM(model=os.environ["GROQ_MODEL_NAME"], api_key=os.environ["GROQ_API_KEY"])

In [36]:
# Loading config files
files = {"agents": "config/agents.yaml", "tasks": "config/tasks.yaml"}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs["agents"]
tasks_config = configs["tasks"]

## Pydantic Model

In [37]:
from typing import List
from pydantic import BaseModel, Field


class OutputModel(BaseModel):
    blog: str = Field(..., description="Detailed Blog")
    seo_keywords: List[str] = Field(..., description="List of SEO keywords")
    citations: List[str] = Field(..., description="List of citations")

## Tool Function

In [38]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun


@tool("DuckDuckGoSearch")
def search_tool(search_query: str) -> str:
    """
    Search the web for citation links based on the search query that will be a blog on a given topic.
    Ensure the search_query is a valid string and is the complete blog.
    """
    return DuckDuckGoSearchRun().run(search_query)

## Crew, Agents, Tasks

In [ ]:
from crewai import Agent, Task, Crew

# Agents
task_expansion_agent = Agent(config=agents_config["topic_expansion_agent"], llm=llm)
seo_strategy_agent = Agent(config=agents_config["seo_strategy_agent"], llm=llm)
research_citation_agent = Agent(
    config=agents_config["research_citation_agent"],
    llm=llm,
)
final_formatter_agent = Agent(config=agents_config["final_formatter_agent"], llm=llm)

# Tasks
create_blog_task = Task(
    config=tasks_config["create_blog_task"], agent=task_expansion_agent
)
find_seo_keywords_task = Task(
    config=tasks_config["find_seo_keywords_task"],
    agent=seo_strategy_agent,
    context=[create_blog_task],
)
reference_finder_task = Task(
    config=tasks_config["reference_finder_task"],
    agent=research_citation_agent,
    context=[create_blog_task],
    # tools=[search_tool],
)
finalize_blog_task = Task(
    config=tasks_config["finalize_blog_task"],
    agent=research_citation_agent,
    context=[create_blog_task, find_seo_keywords_task, reference_finder_task],
    output_pydantic=OutputModel
)

# Crew
crew = Crew(
    agents=[
        task_expansion_agent,
        seo_strategy_agent,
        research_citation_agent,
        final_formatter_agent,
    ],
    tasks=[
        create_blog_task,
        find_seo_keywords_task,
        reference_finder_task,
        finalize_blog_task,
    ],
    verbose=True,
)

In [43]:
inputs = {"input_topic": "Chess"}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 703b0876-cd17-40bb-b9a6-4dba74aa6583                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1329424c-9c2b-45e8-b456-a49cd91ff926
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1329424c-9c2b-45e8-b456-a49cd91ff926
       Status: Executing Task...
    └── 🤖 Agent: The Master of Content Creation, Structure and Topic Expansion
        
            Status: In Progress

# Agent: The Master of Content Creation, Structure and Topic Expansion
## Task: Using the expanded outline your task is to write a complete blog post based on the input_topic: "Chess". The blog must follow a clear structure with an engaging introduction, 3-5 detailed body sections (based on the subtopics), and a strong conclusion. Ensure the blog is written in a natural tone suitable for the target audience.



# Agent: The Master of Content Creation, Structure and Topic Expansion
## Final Answer: 
**The Ultimate Guide to Chess: Unlocking the Secrets of the Game**

**Introduction**

Chess, often referred to as the game of kings, has been a cornerstone of strategy and intellectual stimulation for centuries. With its origins dating back to ancient India in the 6th century AD, chess has evolved into a global phenomenon, captivating the minds of people from all walks of life. Whether you're a beginner looking to learn the basics or a seasoned player aiming to improve your skills, this comp

🚀 Crew: crew
└── 📋 Task: 1329424c-9c2b-45e8-b456-a49cd91ff926
       Status: Executing Task...
    └── 🤖 Agent: The Master of Content Creation, Structure and Topic Expansion
        
            Status: ✅ Completed

...

# Agent: The Diligent Research Assistant and Citation Expert
## Task: Use the outputs from the outline, SEO strategy, and citation research to synthesize a complete blog post on "Chess". Ensure a clean structure, flowing transitions between sections, natural integration of SEO keywords, and properly formatted citation links. The tone should be consistent and aligned with the target audience, and the formatting must match standard blog publishing guidelines (subheadings, bullet points, etc.).



# Agent: The Diligent Research Assistant and Citation Expert
## Final Answer: 
The Ultimate Guide to Chess: Unlocking the Secrets of the Game

Chess, often referred to as the game of kings, has been a cornerstone of strategy and intellectual stimulation for centuries. With its origins dating back to ancient India in the 6th century AD, chess has evolved into a global phenomenon, captivating the minds of people from all walks of life. Whether you're a beginner looking to learn the basics or a sea

🚀 Crew: crew
├── 📋 Task: 1329424c-9c2b-45e8-b456-a49cd91ff926
│      Assigned to: The Master of Content Creation, Structure and Topic Expansion
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Master of Content Creation, Structure and Topic Expansion
│       
│           Status: ✅ Completed
├── 📋 Task: b8f786ae-61cc-408e-86fa-ec298266861e
│      Assigned to: The SEO Optimization Specialist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The SEO Optimization Specialist
│       
│           Status: ✅ Completed
├── 📋 Task: 823a8b5a-9165-4657-aacd-96e67d5c58bb
│      Assigned to: The Diligent Research Assistant and Citation Expert
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Diligent Research Assistant and Citation Expert
│       
│           Status: ✅ Completed
└── 📋 Task: 0e630473-5ae3-4b4f-bcfc-69e4a104a423
       Status: Executing Task...
    └── 🤖 Agent: The Diligent Research Assistant and Citation Expert
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 1329424c-9c2b-45e8-b456-a49cd91ff926
│      Assigned to: The Master of Content Creation, Structure and Topic Expansion
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Master of Content Creation, Structure and Topic Expansion
│       
│           Status: ✅ Completed
├── 📋 Task: b8f786ae-61cc-408e-86fa-ec298266861e
│      Assigned to: The SEO Optimization Specialist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The SEO Optimization Specialist
│       
│           Status: ✅ Completed
├── 📋 Task: 823a8b5a-9165-4657-aacd-96e67d5c58bb
│      Assigned to: The Diligent Research Assistant and Citation Expert
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Diligent Research Assistant and Citation Expert
│       
│           Status: ✅ Completed
└── 📋 Task: 0e630473-5ae3-4b4f-bcfc-69e4a104a423
       Assigned to: The Diligent Research Assistant and Citation Expert
    
       Status: ✅ Completed
    └── 🤖 Agent: The Diligent Research Assistant and Citation Expert
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0e630473-5ae3-4b4f-bcfc-69e4a104a423                                                                     │
│  Agent: The Diligent Research Assistant and Citation Expert                                                     │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 703b0876-cd17-40bb-b9a6-4dba74aa6583                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Output